In [1]:
import datetime
import pandas as pd
import geopandas as gpd

import FireMain, FireTime, FireObj, FireConsts, postprocess, preprocess
from utils import timed

In [2]:
tst = [2020, 9, 5, "AM"]
ted = [2020, 9, 8, "PM"]
region = ("Creek", [-119.5, 36.8, -118.9, 37.7])

list_of_ts = list(FireTime.t_generator(tst, ted))

## Once per landcover file

If your landcover file is not in lat,lon then transform it and resave it. You will use this file a bunch later on.

In [3]:
preprocess.preprocess_landcover()

2024-02-28 21:53:27,414 - FireLog - INFO - func:preprocess_landcover took: 24.95 sec


## Once per region

Preprocess the region to get rid of static flare sources. Save that new "swiss cheese" shape off into a geojson file for later.

In [4]:
preprocess.preprocess_region(region)

2024-02-28 21:53:39,498 - FireLog - INFO - func:preprocess_region took: 3.06 sec


## Once per input file

Next process each monthly file into half day files. First we'll get all the times that are of interest. This could also be done by looking at all the files that exist and seeing which have not been preprocessed yet.

In [5]:
%%time
preprocess.preprocess_monthly_file([2020, 9], "SNPP")

2024-02-28 21:53:42,823 - FireLog - INFO - preprocessing monthly file for 2020-9, SNPP
/home/jovyan/fireatlas_nrt/FireIO.py:184: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  df = pd.read_csv(
2024-02-28 21:55:30,885 - FireLog - INFO - func:preprocess_monthly_file took: 1.80 min


CPU times: user 1min 9s, sys: 2.33 s, total: 1min 12s
Wall time: 1min 48s


Note: "NOAA20" files for this time period are not in the input folder.

## Once per region and t

Do initial filtering and clustering using the preprocessed region and the half day files.

In [3]:
region = preprocess.read_region(region)

2024-02-28 21:58:43,041 - FireLog - INFO - func:read_region took: 1.28 ms


In [4]:
%%time
for t in list_of_ts:
    preprocess.preprocess_region_t(t, sensor="SNPP", region=region)

2024-02-28 21:58:43,380 - FireLog - INFO - filtering and clustering 2020-9-5 AM, SNPP, Creek
2024-02-28 21:58:43,407 - FireLog - INFO - func:read_preprocessed took: 26.60 ms
2024-02-28 21:58:43,409 - FireLog - INFO - func:preprocess_region_t took: 29.30 ms
2024-02-28 21:58:43,409 - FireLog - INFO - filtering and clustering 2020-9-5 PM, SNPP, Creek
2024-02-28 21:58:43,501 - FireLog - INFO - func:read_preprocessed took: 90.87 ms
2024-02-28 21:58:43,503 - FireLog - INFO - func:preprocess_region_t took: 94.00 ms
2024-02-28 21:58:43,504 - FireLog - INFO - filtering and clustering 2020-9-6 AM, SNPP, Creek
2024-02-28 21:58:43,540 - FireLog - INFO - func:read_preprocessed took: 35.21 ms
2024-02-28 21:58:43,541 - FireLog - INFO - func:preprocess_region_t took: 37.43 ms
2024-02-28 21:58:43,542 - FireLog - INFO - filtering and clustering 2020-9-6 PM, SNPP, Creek
2024-02-28 21:58:43,609 - FireLog - INFO - func:read_preprocessed took: 66.69 ms
2024-02-28 21:58:49,110 - FireLog - INFO - func:preproc

CPU times: user 505 ms, sys: 151 ms, total: 656 ms
Wall time: 5.99 s


## Run the fire expansion and merging algorithm

`Fire_Forward` is responsible for reading in the preprocessed data created in the `Ingest` notebook

In [5]:
allfires, allpixels = FireMain.Fire_Forward(tst=tst, ted=ted, sat="SNPP", restart=False, region=region)

2024-02-28 21:58:49,385 - FireLog - INFO - func:read_preprocessed took: 3.47 ms
2024-02-28 21:58:49,389 - FireLog - INFO - func:read_preprocessed took: 3.98 ms
2024-02-28 21:58:49,399 - FireLog - INFO - func:read_preprocessed took: 9.13 ms
2024-02-28 21:58:49,399 - FireLog - INFO - func:read_preprocessed took: 0.04 ms
2024-02-28 21:58:49,408 - FireLog - INFO - func:read_preprocessed took: 7.95 ms
2024-02-28 21:58:49,413 - FireLog - INFO - func:read_preprocessed took: 4.83 ms
2024-02-28 21:58:49,423 - FireLog - INFO - func:read_preprocessed took: 9.74 ms
2024-02-28 21:58:49,428 - FireLog - INFO - func:read_preprocessed took: 4.23 ms
2024-02-28 21:58:49,431 - FireLog - INFO - --------------------
2024-02-28 21:58:49,431 - FireLog - INFO - Fire tracking at [2020, 9, 5, 'AM']
2024-02-28 21:58:49,590 - FireLog - INFO - func:Fire_expand_rtree took: 155.68 ms
2024-02-28 21:58:49,591 - FireLog - INFO - func:Fire_merge_rtree took: 0.93 ms
2024-02-28 21:58:49,593 - FireLog - INFO - func:invalida

We can save these files off in case something goes wrong later

In [6]:
postprocess.save_allpixels(allpixels, ted, region)
postprocess.save_allfires_gdf(allfires.gdf, ted, region)

2024-02-28 20:15:38,125 - FireLog - INFO - func:save_allpixels took: 133.37 ms
2024-02-28 20:15:38,172 - FireLog - INFO - func:save_allfires_gdf took: 45.83 ms


## Write Snapshot files

Start by reading in the outputs from the run:

In [7]:
allpixels = postprocess.read_allpixels(ted, region)
allfires_gdf = postprocess.read_allfires_gdf(ted, region)

2024-02-28 20:15:41,461 - FireLog - INFO - func:read_allpixels took: 25.53 ms
2024-02-28 20:15:41,489 - FireLog - INFO - func:read_allfires_gdf took: 26.70 ms


Then iterate through all of the ts and save the snapshot layers.

In [8]:
%%time
postprocess.save_snapshots(allfires_gdf, region, tst, ted)

2024-02-28 20:15:45,206 - FireLog - INFO - func:save_snapshot_layers took: 169.18 ms
2024-02-28 20:15:45,355 - FireLog - INFO - func:save_snapshot_layers took: 146.61 ms
2024-02-28 20:15:45,535 - FireLog - INFO - func:save_snapshot_layers took: 176.39 ms
2024-02-28 20:15:45,709 - FireLog - INFO - func:save_snapshot_layers took: 171.46 ms
2024-02-28 20:15:45,879 - FireLog - INFO - func:save_snapshot_layers took: 166.08 ms
2024-02-28 20:15:46,023 - FireLog - INFO - func:save_snapshot_layers took: 141.38 ms
2024-02-28 20:15:46,214 - FireLog - INFO - func:save_snapshot_layers took: 187.37 ms
2024-02-28 20:15:46,360 - FireLog - INFO - func:save_snapshot_layers took: 143.32 ms
2024-02-28 20:15:46,361 - FireLog - INFO - func:save_snapshots took: 1.33 sec


CPU times: user 633 ms, sys: 18.1 ms, total: 651 ms
Wall time: 1.33 s


## Write Largefire files

First we will find all the large fires:

In [9]:
large_fires = postprocess.find_largefires(allfires_gdf)

2024-02-28 20:15:50,324 - FireLog - INFO - func:find_largefires took: 4.06 ms


In [10]:
large_fires

array([1])

First we'll use the `allpixels` object to create the `nplist` layer

In [11]:
%%time
postprocess.save_large_fires_nplist(allpixels, region, large_fires)

2024-02-28 20:16:03,221 - FireLog - INFO - func:save_fire_nplist took: 750.25 ms
2024-02-28 20:16:03,222 - FireLog - INFO - func:save_large_fires_nplist took: 753.85 ms


CPU times: user 641 ms, sys: 7.62 ms, total: 648 ms
Wall time: 755 ms


The rest of the layers will be created directly from the `allfires_gdf` and have some merge handling

In [12]:
%%time
postprocess.save_large_fires_layers(allfires_gdf, region, large_fires)

2024-02-28 20:16:04,907 - FireLog - INFO - func:merge_rows took: 15.73 ms


1 rows that potentially need a merge


2024-02-28 20:16:05,248 - FireLog - INFO - func:save_fire_layers took: 340.88 ms
2024-02-28 20:16:05,249 - FireLog - INFO - func:save_large_fires_layers took: 364.40 ms


CPU times: user 253 ms, sys: 50 ms, total: 304 ms
Wall time: 365 ms
